In [1]:
# Carga de librerías

import warnings
warnings.filterwarnings("ignore")

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import textwrap
import os
import re
from datetime import datetime, timedelta


#Ruta del chroma
CHROMA_PATH = "Datos/vector_db_chile_2025"

In [ ]:
#Keys y rutas
api_key = "" # key de Google
Chromadb_path = "Datos/vector_db_chile_2025"  # ruta de ChromaDB


# LLM GEMINI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3,
    google_api_key=api_key
)

# Embeddings y base de datos
emb = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# Base de datos Chroma
db = Chroma(
    persist_directory=Chromadb_path,
    embedding_function=emb,
    collection_name="noticias_chile_2025"
)
# Comprobación básica de la colección Chroma
try:
    total = db._collection.count()
    print(f"Total de documentos en 'noticias_chile_2025': {total}")
    
except Exception as e:
    print("Error al comprobar la base de datos:", e)

# Configuración del RAG
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 25, "fetch_k": 100}
)

# Plantilla del prompt
prompt = ChatPromptTemplate.from_template("""
Eres SophiaChronos, una analista que SOLO puede usar la información
entregada en el bloque "Contexto". No puedes usar conocimiento externo
ni tu memoria de entrenamiento.

REGLAS IMPORTANTES:
- Asume que TODAS las noticias del contexto son reales.
- Si el contexto NO contiene información relevante para responder,
  di: "No encontré noticias sobre ese tema en la base de datos."
- NUNCA inventes hechos.
- NUNCA digas que la fecha no ha ocurrido.
- Responde con estilo periodístico claro.
- Cita siempre: medio | fecha | link.

Contexto:
{context}

Pregunta: {question}

Respuesta (usa SOLO el contexto):
""")

Total de documentos en 'noticias_chile_2025': 35438


In [11]:
# Lista de regiones
REGIONES_CLAVE = [
    "Arica", "Tarapacá", "Antofagasta", "Atacama", "Coquimbo", "La Serena",
    "Valparaíso", "Viña", "Metropolitana", "Santiago", "O'Higgins", "Rancagua",
    "Maule", "Talca", "Ñuble", "Chillán", "Biobío", "Concepción",
    "Araucanía", "Temuco", "Los Ríos", "Valdivia", "Los Lagos", "Puerto Montt",
    "Aysén", "Magallanes", "Punta Arenas"
]

# Modo Hibrido de recuperación de documentos para responder preguntas
def recuperar_documentos_hibridos(pregunta, db_chroma, k_final=25):
    
    # Modo Francotirador: filtra exacto de fecha.
    # Modo Cronológico: Ordena por fecha.
    # Modo Semántico: Ordena por relevancia.
    
    # Detección de fecha exacta
    patron_fecha = r"(\d{1,2})\s+de\s+(septiembre)"
    match_fecha = re.search(patron_fecha, pregunta.lower())
    
    fecha_objetivo = None
    if match_fecha:
        dia = match_fecha.group(1).zfill(2) 
        mes = "09" 
        fecha_objetivo = f"2025-{mes}-{dia}"
        print(f"Buscando fecha exacta {fecha_objetivo}")

    # Deteccion por tiempo relativo
    palabras_clave_tiempo = ["semana", "últim", "reciente", "ayer", "hoy", "días", "mes", "fecha", "septiembre"]
    es_busqueda_temporal = any(p in pregunta.lower() for p in palabras_clave_tiempo)
    
    # Busqueda Masiva Inicial
    print(f"Escaneando 2000 vectores candidatos...")
    docs_candidatos = db_chroma.similarity_search(pregunta, k=2000)
    
    docs_filtrados = []
    
    # Aplicación de filtro por región
    region_objetivo = None
    for region in REGIONES_CLAVE:
        if region.lower() in pregunta.lower():
            region_objetivo = region
            break
            
    if region_objetivo:
        region_clean = region_objetivo.lower()
        print(f"Filtrando por región: '{region_objetivo}'")
        for doc in docs_candidatos:
            meta_regiones = str(doc.metadata.get("regiones", "")).lower()
            texto_contenido = doc.page_content.lower()
            if region_clean in meta_regiones or region_clean in texto_contenido:
                docs_filtrados.append(doc)
    else:
        docs_filtrados = docs_candidatos




    
    docs_finales = []
    
    # Busqueda por Fecha exacta
    # Se le da 1 dia de margen, porque a veces las noticias se publican al dia sgte
    if fecha_objetivo:
        print("Aplicando filtro de fecha exacta") 
        fecha_obj_dt = datetime.strptime(fecha_objetivo, "%Y-%m-%d")
        fecha_next_dt = fecha_obj_dt + timedelta(days=1)
        fecha_next = fecha_next_dt.strftime("%Y-%m-%d")
        
        for doc in docs_filtrados:
            f_doc = doc.metadata.get("fecha", "")
            if f_doc == fecha_objetivo or f_doc == fecha_next:
                docs_finales.append(doc)
        
        if len(docs_finales) > 0:
            print(f" Encontrados {len(docs_finales)} documentos de la fecha exacta.")
            return docs_finales
        else:
            print("No hubo noticias exactas en esa fecha. Volviendo a búsqueda semántica...")
            docs_finales = docs_filtrados

    # Busqueda por orden cronológico (preguntas temporales y por si falla la exacta)
    elif es_busqueda_temporal:
        print("Reordenando por fecha")
        try:
            docs_filtrados.sort(key=lambda x: x.metadata.get("fecha", "0000-00-00"), reverse=True)
            docs_finales = docs_filtrados
        except:
            docs_finales = docs_filtrados
            
    # Busqueda por relevancia
    else:
        print("Aplicando busqueda por relevancia.")
        docs_finales = docs_filtrados

    return docs_finales[:k_final]


# Formateo de string de contexto para el prompt
def formatear(docs, max_chars=1200):
    # Incluye medio, fecha, link y título
    bloques = []
    for doc in docs:
        meta = doc.metadata or {}
        medio = meta.get("medio", "medio desconocido")
        fecha = meta.get("fecha", "fecha desconocida")
        url = meta.get("url", "link no disponible")
        titulo = meta.get("titulo", "Título no disponible")
        contenido = doc.page_content.strip()
        bloques.append(
            f"{medio} | {fecha} | {url}\nTítulo: {titulo}\n{contenido}"
        )
    contexto = "\n\n---\n\n".join(bloques)
    if max_chars and len(contexto) > max_chars:
        contexto = contexto[:max_chars] + " ..."
    return contexto


# Funcion de SophiaChronos
def sophia(pregunta):
    print(f"\nSophiaChronos pensando: '{pregunta}'")
    
    try:
        # Recuperación Manual de Documentos
        docs = recuperar_documentos_hibridos(pregunta, db)
        
        if not docs:
            print(" No hay documentos después del filtro.")
            return

        # Formatear contexto para el prompt
        contexto_str = formatear(docs)
        
        # Generar respuesta
        mensaje_final = prompt.format(context=contexto_str, question=pregunta)
        
        print(" Generando respuesta con Gemini...")
        respuesta = llm.invoke(mensaje_final)
        
        print("="*90)
        print("SOPHIA CHRONOS DICE:")
        print("="*90)
        print(textwrap.fill(respuesta.content, 100))
        print("="*90)
        
    except Exception as e:
        print(f"Error: {e}")

# EJECUTA ESTA PRUEBA ESPECÍFICA
#sophia("¿Qué pasó con Gabriel Boric en septiembre 2025?")
#sophia("Resumen del acuerdo litio entre Codelco y SQM")
#sophia("¿Hubo incendios forestales graves en esa época?")
sophia("¿Qué dijo la oposición sobre la reforma de pensiones?")
#sophia("¿que paso en valdivia la ultima semana de septiembre de 2025? Listar los eventos principales")
#sophia("Que paso en antofagasta el 20 de septiembre en temas de seguridad")
#sophia("Cuantas noticias de seguridad durante el mes de septiembre de 2025?")


SophiaChronos pensando: '¿Qué dijo la oposición sobre la reforma de pensiones?'
Escaneando 2000 vectores candidatos...
Aplicando busqueda por relevancia.
 Generando respuesta con Gemini...
SOPHIA CHRONOS DICE:
Según el medio El Ciudadano, la oposición, junto al oficialismo, ha convertido la "pequeña y
miserable alteración" al sistema de pensiones en una cuestión trascendental. Ambos justifican y
reivindican la necesidad de acuerdos, y se disputan quién puso más en esa historia.  elciudadano |
2025-09-18 | https://www.elciudadano.com/chile/las-afp-tienen-su-pension-asegurada/09/18/
